# Домашнее задание № 7

## Задание 1 (4 балла)

Обучите 2 модели похожую по архитектуре на модель из ULMFit для задачи классификации текста (датасет - lenta_40k )
В моделях должно быть как минимум два рекуррентных слоя, а финальный вектор для классификации составляться из последнего состояния RNN (так делалось в семинаре), а также AveragePooling и MaxPooling из всех векторов последовательности (конкатенируйте последнее состояния и результаты пулинга). В первой модели используйте обычные слои, а во второй Bidirectional. Рассчитайте по классовую точность/полноту/f-меру для каждой из модели (результаты не должны быть совсем близкие к нулю после обучения на хотя бы нескольких эпохах).

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, TensorDataset
import pandas as pd
import numpy as np
from string import punctuation
from sklearn.model_selection import train_test_split
from collections import Counter
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.metrics import precision_score, recall_score, classification_report, f1_score
from tqdm import tqdm

In [ ]:
data = pd.read_csv(r'sample_data/lenta_40k.csv')

In [ ]:
def preprocess(text):
    tokens = text.lower().split()
    tokens = [token.strip(punctuation) for token in tokens]
    return tokens

vocab = Counter()

for text in data.text:
    vocab.update(preprocess(text))

len(vocab)

filtered_vocab = set()

for word in vocab:
    if vocab[word] > 30:
        filtered_vocab.add(word)

len(filtered_vocab)

word2id = {'PAD': 0, 'UNK': 1}

for word in filtered_vocab:
    word2id[word] = len(word2id)

id2word = {i: word for word, i in word2id.items()}

X = []

for text in data.text:
    tokens = preprocess(text)
    ids = [word2id.get(token, 1) for token in tokens]  # 1 = UNK
    X.append(ids)

MAX_LEN = max(len(x) for x in X)
MEAN_LEN = np.median([len(x) for x in X])
MAX_LEN, MEAN_LEN

MAX_LEN = int(MEAN_LEN + 30)

def pad_sequence(seq, max_len, pad_value=0):
    if len(seq) > max_len:
        return seq[:max_len]
    return seq + [pad_value] * (max_len - len(seq))

X = np.array([pad_sequence(x, MAX_LEN) for x in X])
X.shape

id2label = {i: label for i, label in enumerate(set(data.topic.values))}
label2id = {l: i for i, l in id2label.items()}
y = np.array([label2id[label] for label in data.topic.values])
len(label2id)

19

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.05, stratify=y)

X_train = torch.LongTensor(X_train)
X_valid = torch.LongTensor(X_valid)
y_train = torch.LongTensor(y_train)
y_valid = torch.LongTensor(y_valid)

BATCH_SIZE = 256
train_loader = DataLoader(TensorDataset(X_train, y_train), batch_size=BATCH_SIZE, shuffle=True)
valid_loader = DataLoader(TensorDataset(X_valid, y_valid), batch_size=BATCH_SIZE, shuffle=False)

In [ ]:
class ULMFitClassifier(nn.Module):
    def __init__(self, vocab_size, emb_dim, hidden_size, output_dim, dropout=0.3, bidirectional=False):
        super().__init__()

        self.embedding = nn.Embedding(vocab_size, emb_dim, padding_idx=0)
        self.rnn = nn.LSTM(
            emb_dim,
            hidden_size,
            num_layers=2,
            batch_first=True,
            dropout=dropout,
            bidirectional=bidirectional
        )

        mult = 2 if bidirectional else 1

        self.fc = nn.Linear(hidden_size * mult * 3, output_dim)

        self.dropout = nn.Dropout(dropout)

    def forward(self, text):
        embedded = self.dropout(self.embedding(text))
        output, (hidden, cell) = self.rnn(embedded)

        last_hidden = output[:, -1, :]
        avg_pool = torch.mean(output, dim=1)
        max_pool, _ = torch.max(output, dim=1)
        cat_vector = torch.cat((last_hidden, avg_pool, max_pool), dim=1)

        return self.fc(self.dropout(cat_vector))

In [ ]:
def train(model, iterator, optimizer, criterion):
    model.train()
    epoch_loss = 0

    for texts, labels in tqdm(iterator, desc="Training", leave=False):

        optimizer.zero_grad()

        predictions = model(texts)

        loss = criterion(predictions, labels)

        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    return epoch_loss / len(iterator)

def evaluate(model, iterator, criterion):
    model.eval()
    epoch_loss = 0

    all_preds = []
    all_labels = []

    with torch.no_grad():
        for texts, labels in tqdm(iterator, desc="Evaluating", leave=False):

            predictions = model(texts)
            loss = criterion(predictions, labels)
            epoch_loss += loss.item()

            preds = predictions.argmax(dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    f1 = f1_score(all_labels, all_preds, average='weighted')

    target_labels = list(id2label.keys())
    target_names = [id2label[i] for i in target_labels]

    report = classification_report(
        all_labels,
        all_preds,
        labels=target_labels,
        target_names=target_names,
        zero_division=0
    )

    return epoch_loss / len(iterator), f1, report

In [ ]:
model_rnn = ULMFitClassifier(
    vocab_size=len(word2id),
    emb_dim=300,
    hidden_size=256,
    output_dim=len(label2id),
    bidirectional=False
)

optimizer = optim.Adam(model_rnn.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss()

N_EPOCHS = 3

for epoch in range(N_EPOCHS):
    train_loss = train(model_rnn, train_loader, optimizer, criterion)
    valid_loss, valid_f1, report = evaluate(model_rnn, valid_loader, criterion)

    print(f'Epoch: {epoch+1:02}')
    print(f'\tTrain Loss: {train_loss:.3f}')
    print(f'\tVal. Loss: {valid_loss:.3f} | Val. F1 (weighted): {valid_f1:.3f}')

    if epoch == N_EPOCHS - 1:
        print("\nClassification Report")
        print(report)

Epoch: 01
	Train Loss: 2.023
	Val. Loss: 1.468 | Val. F1 (weighted): 0.482


Epoch: 02
	Train Loss: 1.292
	Val. Loss: 1.098 | Val. F1 (weighted): 0.635


Epoch: 03
	Train Loss: 1.015
	Val. Loss: 0.920 | Val. F1 (weighted): 0.690

Classification Report
                   precision    recall  f1-score   support

       Библиотека       0.00      0.00      0.00         0
              Мир       0.74      0.81      0.78       410
  Наука и техника       0.73      0.76      0.74       160
        Экономика       0.80      0.75      0.77       239
      Бывший СССР       0.80      0.77      0.79       159
    Культпросвет        0.00      0.00      0.00         1
         Из жизни       0.45      0.42      0.43        84
   69-я параллель       0.00      0.00      0.00         4
Силовые структуры       0.67      0.03      0.06        60
   Интернет и СМИ       0.46      0.17      0.25       132
      Путешествия       0.00      0.00      0.00        21
              Дом       0.57      0.68      0.62        66
           Бизнес       0.00      0.00      0.00        22
          Легпром       0.00      0.00      0.00         0
             Кры

In [ ]:
model_birnn = ULMFitClassifier(
    vocab_size=len(word2id),
    emb_dim=300,
    hidden_size=256,
    output_dim=len(label2id),
    bidirectional=True
)

optimizer_bi = optim.Adam(model_birnn.parameters(), lr=1e-3)

criterion = nn.CrossEntropyLoss()

N_EPOCHS = 3

for epoch in range(N_EPOCHS):
    train_loss = train(model_birnn, train_loader, optimizer_bi, criterion)
    valid_loss, valid_f1, report = evaluate(model_birnn, valid_loader, criterion)

    print(f'Epoch: {epoch+1:02}')
    print(f'\tTrain Loss: {train_loss:.3f}')
    print(f'\tVal. Loss: {valid_loss:.3f} | Val. F1 (weighted): {valid_f1:.3f}')

    # Выводим отчет на последней эпохе
    if epoch == N_EPOCHS - 1:
        print("\nClassification Report (Bidirectional RNN)")
        print(report)

Epoch: 01
	Train Loss: 1.798
	Val. Loss: 1.179 | Val. F1 (weighted): 0.591


Epoch: 02
	Train Loss: 1.100
	Val. Loss: 0.923 | Val. F1 (weighted): 0.683


Epoch: 03
	Train Loss: 0.858
	Val. Loss: 0.762 | Val. F1 (weighted): 0.746

Classification Report (Bidirectional RNN)
                   precision    recall  f1-score   support

       Библиотека       0.00      0.00      0.00         0
              Мир       0.75      0.84      0.79       410
  Наука и техника       0.75      0.80      0.77       160
        Экономика       0.82      0.79      0.81       239
      Бывший СССР       0.76      0.89      0.82       159
    Культпросвет        0.00      0.00      0.00         1
         Из жизни       0.48      0.51      0.49        84
   69-я параллель       0.00      0.00      0.00         4
Силовые структуры       0.61      0.50      0.55        60
   Интернет и СМИ       0.57      0.26      0.35       132
      Путешествия       0.67      0.10      0.17        21
              Дом       0.56      0.74      0.64        66
           Бизнес       0.14      0.05      0.07        22
          Легпром       0.00      0.00      0.00       

Двунаправленная модель показала лучший результат!

## Задание 2 (6 баллов)


На данных википедии (wikiann) обучите и сравните 3 модели:  
1) модель в которой как минимум два рекуррентных слоя, причем один из них GRU, а другой LSTM
2) модель в которой как минимум 3 рекуррентных слоя идут друг за другом и при этом 2-ой и 3-й слои еще имеют residual connection к изначальным эмбедингам. Для того, чтобы сделать residual connection вам нужно будет использовать одинаковую размерность эмбедингов и количество unit'ов в RNN слоях, чтобы их можно было просуммировать
3) модель в которой будут и рекуррентные и сверточные слои (как минимум 2 rnn и как минимум 2 cnn слоя). В cnn слоях будьте аккуратны с укорачиванием последовательности и используйте паддинг



Сравните качество по метрикам (точность/полнота/f-мера). Также придумайте несколько сложных примеров и проверьте, какие сущности определяет каждая из моделей.

In [6]:
from datasets import load_dataset

dataset = load_dataset("wikiann", "ru")

ner_tags = dataset['train'].features['ner_tags'].feature.names
print("Теги сущностей:", ner_tags)

README.md: 0.00B [00:00, ?B/s]

ru/validation-00000-of-00001.parquet:   0%|          | 0.00/809k [00:00<?, ?B/s]

ru/test-00000-of-00001.parquet:   0%|          | 0.00/816k [00:00<?, ?B/s]

ru/train-00000-of-00001.parquet:   0%|          | 0.00/1.63M [00:00<?, ?B/s]

Generating validation split:   0%|          | 0/10000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/10000 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/20000 [00:00<?, ? examples/s]

Теги сущностей: ['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC']


In [7]:
vocab = Counter()
for tokens in dataset['train']['tokens']:
    vocab.update([t.lower() for t in tokens])

filtered_vocab = {word for word, count in vocab.items() if count >= 3}

word2id = {'PAD': 0, 'UNK': 1}
for word in filtered_vocab:
    word2id[word] = len(word2id)

id2word = {i: word for word, i in word2id.items()}
print(f"Размер словаря: {len(word2id)}")

def preprocess_data(data_split):
    X = []
    y = []

    for item in data_split:
        tokens = item['tokens']
        tags = item['ner_tags']

        token_ids = [word2id.get(t.lower(), 1) for t in tokens] # 1 = UNK

        X.append(token_ids)
        y.append(tags)

    return X, y

X_train_raw, y_train_raw = preprocess_data(dataset['train'])
X_valid_raw, y_valid_raw = preprocess_data(dataset['validation'])
X_test_raw, y_test_raw = preprocess_data(dataset['test'])

MAX_LEN = 50

def pad_sequences(seqs, max_len, pad_value):
    padded = []
    for seq in seqs:
        if len(seq) > max_len:
            padded.append(seq[:max_len])
        else:
            padded.append(seq + [pad_value] * (max_len - len(seq)))
    return np.array(padded)

X_train = pad_sequences(X_train_raw, MAX_LEN, 0)
y_train = pad_sequences(y_train_raw, MAX_LEN, -100)

X_valid = pad_sequences(X_valid_raw, MAX_LEN, 0)
y_valid = pad_sequences(y_valid_raw, MAX_LEN, -100)

X_test = pad_sequences(X_test_raw, MAX_LEN, 0)
y_test = pad_sequences(y_test_raw, MAX_LEN, -100)

BATCH_SIZE = 128

train_dataset = TensorDataset(torch.LongTensor(X_train), torch.LongTensor(y_train))
valid_dataset = TensorDataset(torch.LongTensor(X_valid), torch.LongTensor(y_valid))
test_dataset = TensorDataset(torch.LongTensor(X_test), torch.LongTensor(y_test))

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

print(f"X_train: {X_train.shape}, y_train: {y_train.shape}")

Размер словаря: 5326
X_train: (20000, 50), y_train: (20000, 50)


In [12]:
class NER_RNN_Model(nn.Module):
    def __init__(self, vocab_size, emb_dim, hidden_size, output_dim, dropout=0.2):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, emb_dim, padding_idx=0)

        self.gru = nn.GRU(emb_dim, hidden_size, batch_first=True, bidirectional=True)

        self.lstm = nn.LSTM(hidden_size * 2, hidden_size, batch_first=True, bidirectional=True)

        self.fc = nn.Linear(hidden_size * 2, output_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, text):
        embedded = self.dropout(self.embedding(text))

        gru_out, _ = self.gru(embedded)

        # Исправлено: используем lstm_out
        lstm_out, _ = self.lstm(self.dropout(gru_out))

        return self.fc(self.dropout(lstm_out))

In [13]:
def train_ner(model, iterator, optimizer, criterion):
    model.train()
    epoch_loss = 0
    for texts, labels in tqdm(iterator, desc="Training NER"):
        optimizer.zero_grad()

        outputs = model(texts) # [B, L, C]

        outputs = outputs.view(-1, outputs.shape[-1])
        labels = labels.view(-1)

        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    return epoch_loss / len(iterator)

def evaluate_ner(model, iterator, criterion, target_names):
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for texts, labels in iterator:
            outputs = model(texts)
            preds = outputs.argmax(dim=-1).cpu().numpy()
            labels = labels.cpu().numpy()

            for i in range(labels.shape[0]):
                for j in range(labels.shape[1]):
                    if labels[i, j] != -100:
                        all_preds.append(preds[i, j])
                        all_labels.append(labels[i, j])

    report = classification_report(all_labels, all_preds, target_names=target_names, zero_division=0)
    return report

In [14]:
model1 = NER_RNN_Model(len(word2id), 128, 128, len(ner_tags))
optimizer1 = optim.Adam(model1.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss(ignore_index=-100)

for epoch in range(3): # 3 эпохи для примера
    loss = train_ner(model1, train_loader, optimizer1, criterion)
    print(f"Epoch {epoch+1}, Loss: {loss:.4f}")

report1 = evaluate_ner(model1, valid_loader, criterion, ner_tags)
print(report1)

Training NER: 100%|██████████| 157/157 [01:58<00:00,  1.33it/s]


Epoch 1, Loss: 0.8383


Training NER: 100%|██████████| 157/157 [02:01<00:00,  1.29it/s]


Epoch 2, Loss: 0.5155


Training NER: 100%|██████████| 157/157 [01:57<00:00,  1.33it/s]


Epoch 3, Loss: 0.4303
              precision    recall  f1-score   support

           O       0.88      0.97      0.93     39962
       B-PER       0.92      0.79      0.85      3590
       I-PER       0.94      0.88      0.91      7570
       B-ORG       0.75      0.50      0.60      3891
       I-ORG       0.82      0.70      0.75      7297
       B-LOC       0.75      0.72      0.73      4849
       I-LOC       0.82      0.69      0.75      3106

    accuracy                           0.87     70265
   macro avg       0.84      0.75      0.79     70265
weighted avg       0.87      0.87      0.86     70265



In [16]:
class NER_Residual_RNN(nn.Module):
    def __init__(self, vocab_size, emb_dim, output_dim, dropout=0.2):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, emb_dim, padding_idx=0)

        self.rnn1 = nn.GRU(emb_dim, emb_dim, batch_first=True)
        self.rnn2 = nn.GRU(emb_dim, emb_dim, batch_first=True)
        self.rnn3 = nn.GRU(emb_dim, emb_dim, batch_first=True)

        self.fc = nn.Linear(emb_dim, output_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, text):
        emb = self.dropout(self.embedding(text)) # [B, L, E]

        out1, _ = self.rnn1(emb)

        out2, _ = self.rnn2(self.dropout(out1))
        out2 = out2 + emb

        out3, _ = self.rnn3(self.dropout(out2))
        out3 = out3 + emb

        return self.fc(self.dropout(out3))

In [18]:
model2 = NER_Residual_RNN(len(word2id), 256, len(ner_tags))
optimizer2 = optim.Adam(model2.parameters(), lr=1e-3)

for epoch in range(3):
    loss = train_ner(model2, train_loader, optimizer2, criterion)
    print(f"Epoch {epoch+1}, Loss: {loss:.4f}")

report2 = evaluate_ner(model2, valid_loader, criterion, ner_tags)
print(report2)

Training NER: 100%|██████████| 157/157 [03:06<00:00,  1.19s/it]


Epoch 1, Loss: 0.9301


Training NER: 100%|██████████| 157/157 [03:02<00:00,  1.16s/it]


Epoch 2, Loss: 0.6591


Training NER: 100%|██████████| 157/157 [02:48<00:00,  1.07s/it]


Epoch 3, Loss: 0.5530
              precision    recall  f1-score   support

           O       0.86      0.95      0.91     39962
       B-PER       0.54      0.77      0.64      3590
       I-PER       0.90      0.86      0.88      7570
       B-ORG       0.70      0.35      0.46      3891
       I-ORG       0.77      0.70      0.74      7297
       B-LOC       0.83      0.46      0.60      4849
       I-LOC       0.83      0.62      0.71      3106

    accuracy                           0.83     70265
   macro avg       0.78      0.67      0.70     70265
weighted avg       0.83      0.83      0.82     70265



In [19]:
class NER_CNN_RNN(nn.Module):
    def __init__(self, vocab_size, emb_dim, hidden_size, output_dim, dropout=0.2):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, emb_dim, padding_idx=0)

        self.conv1 = nn.Conv1d(emb_dim, emb_dim, kernel_size=3, padding=1)
        self.conv2 = nn.Conv1d(emb_dim, emb_dim, kernel_size=3, padding=1)

        self.rnn = nn.LSTM(emb_dim, hidden_size, num_layers=2,
                           batch_first=True, bidirectional=True, dropout=dropout)

        self.fc = nn.Linear(hidden_size * 2, output_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, text):
        embedded = self.embedding(text)

        x = embedded.transpose(1, 2)

        x = F.relu(self.conv1(self.dropout(x)))
        x = F.relu(self.conv2(self.dropout(x)))

        x = x.transpose(1, 2)

        rnn_out, _ = self.rnn(x)

        return self.fc(self.dropout(rnn_out))

In [20]:
model3 = NER_CNN_RNN(len(word2id), 128, 128, len(ner_tags))
optimizer3 = optim.Adam(model3.parameters(), lr=1e-3)

for epoch in range(3):
    loss = train_ner(model3, train_loader, optimizer3, criterion)
    print(f"Epoch {epoch+1}, Loss: {loss:.4f}")

report3 = evaluate_ner(model3, valid_loader, criterion, ner_tags)
print(report3)

Training NER: 100%|██████████| 157/157 [02:26<00:00,  1.07it/s]


Epoch 1, Loss: 0.9635


Training NER: 100%|██████████| 157/157 [02:22<00:00,  1.10it/s]


Epoch 2, Loss: 0.5609


Training NER: 100%|██████████| 157/157 [02:23<00:00,  1.09it/s]


Epoch 3, Loss: 0.4783
              precision    recall  f1-score   support

           O       0.90      0.96      0.93     39962
       B-PER       0.94      0.76      0.84      3590
       I-PER       0.93      0.87      0.90      7570
       B-ORG       0.73      0.44      0.55      3891
       I-ORG       0.75      0.72      0.73      7297
       B-LOC       0.69      0.73      0.71      4849
       I-LOC       0.68      0.69      0.69      3106

    accuracy                           0.86     70265
   macro avg       0.80      0.74      0.76     70265
weighted avg       0.85      0.86      0.85     70265



In [21]:
def predict_tags(text, model, word2id, id2label):
    model.eval()
    tokens = text.lower().split()
    ids = [word2id.get(t, 1) for t in tokens]
    input_tensor = torch.LongTensor([ids])

    with torch.no_grad():
        output = model(input_tensor)
        preds = output.argmax(dim=-1)[0].cpu().numpy()

    return list(zip(tokens, [id2label[p] for p in preds]))

examples = [
    "Студия Лебедева производит собственные шрифты с 2004 года. Ими пользуются компании всех направлений и размеров.",
    "До конца марта владельцами сети Братья Караваевы (более 40 кулинарий) были Евгений Каценельсон и Игорь Моисеев (им обоим принадлежало по 50% ООО «Селена»). Новым собственником стал торговый дом Нефтьмагистраль — топливная компания, которой принадлежит сеть АЗС.",
    "Нью-Йорк Сити — футбольный клуб из США."
]

id2label = {i: name for i, name in enumerate(ner_tags)}

for text in examples:
    print(f"\nТекст: {text}")
    print("Модель 1 (RNN):", predict_tags(text, model1, word2id, id2label))
    print("Модель 2 (Res):", predict_tags(text, model2, word2id, id2label))
    print("Модель 3 (Hybrid):", predict_tags(text, model3, word2id, id2label))


Текст: Студия Лебедева производит собственные шрифты с 2004 года. Ими пользуются компании всех направлений и размеров.
Модель 1 (RNN): [('студия', 'O'), ('лебедева', 'O'), ('производит', 'O'), ('собственные', 'O'), ('шрифты', 'O'), ('с', 'O'), ('2004', 'O'), ('года.', 'O'), ('ими', 'O'), ('пользуются', 'O'), ('компании', 'O'), ('всех', 'O'), ('направлений', 'B-LOC'), ('и', 'O'), ('размеров.', 'O')]
Модель 2 (Res): [('студия', 'O'), ('лебедева', 'O'), ('производит', 'O'), ('собственные', 'O'), ('шрифты', 'O'), ('с', 'O'), ('2004', 'B-ORG'), ('года.', 'O'), ('ими', 'O'), ('пользуются', 'O'), ('компании', 'O'), ('всех', 'O'), ('направлений', 'O'), ('и', 'O'), ('размеров.', 'O')]
Модель 3 (Hybrid): [('студия', 'O'), ('лебедева', 'O'), ('производит', 'O'), ('собственные', 'O'), ('шрифты', 'O'), ('с', 'O'), ('2004', 'O'), ('года.', 'O'), ('ими', 'O'), ('пользуются', 'O'), ('компании', 'O'), ('всех', 'O'), ('направлений', 'O'), ('и', 'O'), ('размеров.', 'O')]

Текст: До конца марта владельца

Ввиду недостатка времени самой лучшей по метрикам оказалась первая модель. Возможно, дообучение исправило бы ситуацию. В примерах модели часто не фиксируют сущности, тегируют как O, как в случае со Студией Лебедева.